In [130]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown

## Set up

1. (*Feasibility*) $\mu_{ij} \in \{0,1\}$ and $\sum_{i} \mu_{ij} \leq 1$ and $\sum_{j} \mu_{ij} \leq 1 $ for each $ij \in \mathcal{I}\times\mathcal{J},$
2. (*Generalized Complementary Slackness*) $D_{ij}(u_i,v_j) \geq 0$ with equality if $\mu_{ij}  = 1$  $(\forall ij)$, 
3. (*Individual Rationality*) $u_i \geq u_{i0},$ $ v_j \geq v_{0j}$ for each $ij \in \mathcal{I}\times\mathcal{J},$ with equality if $\sum_{j} \mu_{ij} = 0 \ (\text{ resp. } \sum_{i} \mu_{ij} = 0)$



We try to solve the case with
$$D_{ij}(U_i, V_j) = A_{ij} U_i + G_{ij} V_j - B_{ij}$$
First we define our problem (setting $U_{i0} = V_{0j} = 1$ by default):

In [171]:
class OneToOneITU():
  def __init__(self, n, m, size_params , random_seed, lbs = (1,1), tol = 1e-9):
    
    self.n = n
    self.m = m
    self.lb_U , self.lb_V = lbs
    self.tol = tol 

    #generate problem
    np.random.seed(random_seed)

    self.A_ij = np.random.randint(1,size_params[0] ,size = [n,m])
    self.B_ij = np.random.randint(0,size_params[1] ,size = [n,m])

  def D_ij(self, U_i, V_j):
    return U_i[:,None]  +  self.A_ij * V_j[None,:] - self.B_ij

    
    
    

#### Checking modules

The following functions check equilibrium conditions for a candidate $(\mu, U, V).$. Below I will refer to the equilibrium conditions as follows:
* (*Primal Feasibility*) $\sum_{i} \mu_{ij} \leq 1$ and $\sum_{j} \mu_{ij} \leq 1 $ (and $\mu_{ij} \in \{0,1\}$),
* (*Primal Complementary Slackness*) $D_{ij}(U_i,V_j) = 0$ if $ \mu_{ij}  = 1$, 
* (*Dual Feasibility*) $D_{ij}(U_i,V_j) \geq 0$,
* (*Dual Complementary Slackness*) $U_i =  U_0 $ if $\sum_j \mu_{ij} = 0$ and $V_i =  V_0 $ if $\sum_i \mu_{ij} = 0,$ 
* (*Individual Rationality*) $U_i \geq  U_0 ,$ $V_i \geq  V_0 .$ 

**Dual Complementary Slackness**: $\forall i,j, \ U_i > U_{0} \implies \sum_j \mu_{ij} = 0, \ V_j > V_{0} \implies \sum_i \mu_{ij} = 0$

In [172]:
def check_dual_CS(self,U_i,V_j,mu_ij, output = False):

    D_CS_i = np.mean((U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1))
    D_CS_j = np.mean((V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1)) 
    display(Math(fr'\frac{1}{{n}} \sum_{{i}} (U_{{i}} - U_{{lb}} ) * (\sum_{{j}} \mu_{{ij}} -1) = {D_CS_i}'))
    display(Math(fr'\frac{1}{{m}}\sum_{{j}} (V_{{j}} - V_{{lb}} ) * (\sum_{{i}} \mu_{{ij}} -1) = {D_CS_j}'))
    
    if output is True:
        D_CS_i_bool = (U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1) >= 0 
        D_CS_j_bool = (V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1) >= 0
        return D_CS_i_bool, D_CS_j_bool
    
OneToOneITU.check_dual_CS = check_dual_CS

**Dual Feasibility**:  $\forall i,j,  \ A_{ij} U_i + G_{ij} V_j \geq B_{ij},$

In [173]:
def check_dual_feas(self,U_i,V_j, output = None):

    D_feas = np.minimum(self.D_ij(U_i,V_j),0)
    display(Math(fr'\frac{1}{{nm}} \sum_{{ij}} \min( A_{{ij}} U_i + G_{{ij}} V_j -  B_{{ij}} , 0)  = {D_feas.mean()}'))
    if output:
        return D_feas, np.where(D_feas<0 )
   
OneToOneITU.check_dual_feas = check_dual_feas 

**Primal Complementary Slackness**:  $\forall i,j, \  \mu_{ij} * (A_{ij} U_i + G_{ij} V_j - B_{ij}) = 0 $

In [174]:
def check_primal_CS(self, U_i,V_j,mu_ij, output = None):

    P_CS_ij = self.D_ij(U_i,V_j) * mu_ij
    display(Math(fr'\frac{1}{{nm}} \sum_{{ij}} \mu_{{ij}} * (A_{{ij}} U_i + G_{{ij}} V_j - B_{{ij}}) = {P_CS_ij.mean()}'))
    if output:
        return P_CS_ij

OneToOneITU.check_primal_CS = check_primal_CS

**Primal Feasibility**:  $\forall i,j, \  \sum_j  \mu_{ij} \leq 1, \ \sum_i  \mu_{ij} \leq 1$

In [175]:
def check_primal_feas(self, mu_ij):

    display(Math(fr'\forall i, \ \sum_{{j}}  \mu_{{ij}} \leq 1: \ { np.all(np.sum(mu_ij,axis=1) <= np.ones(self.n))}'))
    display(Math(fr'\forall j, \ \sum_{{i}}  \mu_{{ij}} \leq 1: \ {np.all(np.sum(mu_ij,axis=0) <= np.ones(self.m)) }'))
    display(Markdown(f"#matched: {int(np.sum(mu_ij))} over {np.minimum(self.n,self.m)}"))

OneToOneITU.check_primal_feas = check_primal_feas

In [176]:
def check_IR(self, U_i,V_j):
    display(Math(fr'\min_{{i}} U_{{i}} = {np.min(U_i).round(3)} \geq {self.lb_U} = U_{{lb}}'))
    display(Math(fr'\min_{{j}} V_{{j}} = {np.min(V_j).round(3)} \geq  {self.lb_V} = V_{{lb}}'))

OneToOneITU.check_IR = check_IR

The following checks all conditions and prints the results

In [177]:
def check_all(self,eq):
    mu_ij, U_i, V_j = eq 
    header_size = 1

    display(Markdown(f"{'_' * 4}\n<h{header_size}>Feasibility</h{header_size}>"))
    self.check_primal_feas(mu_ij)

    display(Markdown(f"{'_' * 4}\n<h{header_size}>Generalized Complementary Slackness</h{header_size}>"))
    self.check_dual_feas(U_i, V_j )
    self.check_primal_CS(U_i, V_j , mu_ij)
    
    display(Markdown(f"{'_' * 4}\n<h{header_size}>Individual Rationality</h{header_size}>"))
    self.check_IR(U_i, V_j )
    self.check_dual_CS(U_i, V_j , mu_ij)

OneToOneITU.check_all = check_all

## Algorithm

In [178]:
def find_new_entering(self, i_ent, mu_ij, U_i,V_j, print_output = False):

    U_i_ent =  (np.maximum((self.B_ij[i_ent] -  (self.A_ij[i_ent] * V_j)), self.lb_U))
    sorted_U_ent_id = np.argsort(U_i_ent)[::-1]
    sorted_U_ent = U_i_ent[sorted_U_ent_id]
 
    if sorted_U_ent[0] <= self.lb_U: 
        if print_output:
            display(Math(fr"i_{{ent}} = {i_ent} \text{{ is unmatched}}"))
        U_i[i_ent] = self.lb_U
        return (mu_ij, U_i,V_j), None
    
    else:
        j_pivot = sorted_U_ent_id[0]
        i_dep = mu_ij[j_pivot]
        if i_dep == (self.n + 1): #j_pivot unmatched
            mu_ij[j_pivot] = i_ent
            
            V_j[j_pivot] = (self.B_ij[i_ent,j_pivot] -  sorted_U_ent[1]) / self.A_ij[i_ent,j_pivot] + self.tol
            U_i[i_ent] = self.B_ij[i_ent,j_pivot] -  self.A_ij[i_ent,j_pivot] * V_j[j_pivot]
        
            if print_output:
                display(Math(fr"i_{{ent}} = {i_ent} \text{{ matches (unmatched) }} j_{{pivot}} = {j_pivot} \text{{ with }}"
                             fr"U_{i_ent} = {U_i[i_ent]:.4f}, \ V_{j_pivot} = {V_j[j_pivot]:.4f}"))
            new_vars = None
            return (mu_ij, U_i, V_j) , new_vars

        # second best option for i_ent
        out_ent = sorted_U_ent[1] 
        
        # find second best option for i_dep
        U_i_dep =  (np.maximum(self.B_ij[i_dep] -  (self.A_ij[i_dep] * V_j), self.lb_U))
        sorted_U_dep_id = np.argsort(U_i_dep)[::-1]
        sorted_U_dep = U_i_dep[sorted_U_dep_id]
        out_dep = sorted_U_dep[1]

        ### compute new entering
        V_tilde =  ((self.B_ij[[i_ent,i_dep],j_pivot].flatten() -  np.array([[out_ent,out_dep]])) 
                        / self.A_ij[[i_ent,i_dep],j_pivot].flatten())[0]
        V_tilde_sort_id = np.argsort(V_tilde)
    
        #if V_tilde_sort_id[0] == 0: # i_new_ent = i_ent
        if V_tilde[0] < V_tilde[1]: #i_dep has best offer
            i_ent_new = i_ent
            i_matched = i_dep
            V_j[j_pivot] = V_tilde[1] + self.tol
            if print_output:
                display(Math(fr"i_{{dep}} = {i_dep} \text{{ still matches }} j_{{pivot}} = {j_pivot} \text{{ with}}"))

        #elif V_tilde_sort_id[0] == 1: # i_new_ent = i_dep
        else:
            i_ent_new = i_dep
            i_matched = i_ent
            mu_ij[j_pivot] = i_ent

            V_j[j_pivot] =  V_tilde[0] + self.tol #np.maximum( V_tilde[0] - V_j[j_pivot] ,  self.tol)
            
            if print_output:
                display(Math(fr"i_{{ent}} = {i_ent} \text{{ matches }} j_{{pivot}} = {j_pivot} " 
                             fr"\text{{ (replacing }} i_{{dep}} = {i_dep}\text{{) with}}"))
        # else:
        #     print("something's off")

        # INCREASE the value of j_pivot to v_tilde_j (the maximum of the two)
        #V_j[j_pivot] += np.maximum( V_tilde[V_tilde_sort_id[1]] - V_j[j_pivot] ,  self.tol)
        
        # update the value of new matched agent in order to give j value v_tilde_j, by construction U_imatched > 0
        U_i[i_matched] = self.B_ij[i_matched,j_pivot] - (self.A_ij[i_matched,j_pivot] * V_j[j_pivot])
        if print_output:
            display(Math(fr"U_{i_matched} = {U_i[i_matched].round(4)}, \  V_{j_pivot} = {V_j[j_pivot].round(4)}, " 
                         fr"\ \ i_{{entnew}} = {i_ent_new}"))
            print("___________")
        
        return (mu_ij, U_i,V_j) , i_ent_new

OneToOneITU.find_new_entering =find_new_entering

In [179]:
def find_eq_t(self,t,mu_ij, U_i, V_j,print_output):
        i_ent = t
        (mu_ij, U_i ,V_j ), i_ent = self.find_new_entering(i_ent, mu_ij, U_i, V_j,print_output)
        
        while i_ent is not None:
            i_ent = i_ent
            (mu_ij, U_i ,V_j ), i_ent = self.find_new_entering(i_ent, mu_ij, U_i, V_j,print_output)

        return (mu_ij, U_i ,V_j )

OneToOneITU.find_eq_t = find_eq_t

In [180]:
def find_eq(self, print_output = False):

    mu_ij = np.ones(self.m, dtype= int) * (self.n + 1)
    U_i = np.ones(self.n)
    V_j = np.ones(self.m) * self.lb_V
    U_i *= np.inf

    for t in range(self.n):
        if print_output:
            print("__________________________________________________________________")
            print(f"PROBLEM t = {t}")
           
        mu_ij, U_i, V_j = self.find_eq_t(t, mu_ij , U_i, V_j, print_output)

    mu_matrix = np.zeros([self.n, self.m], dtype= bool)
    for j in range(self.m):
        if mu_ij[j]< (self.n+1): 
            mu_matrix[mu_ij[j],j] = 1 



    return mu_matrix, U_i, V_j

OneToOneITU.find_eq = find_eq

## Examples

### Small example

In [181]:
n_small = 6
m_small = 10
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (10,10) , random_seed= 88988, lbs =(2,1))

In [182]:
mu_small , U_small, V_small = example_small.find_eq( print_output= True)

__________________________________________________________________
PROBLEM t = 0


<IPython.core.display.Math object>

__________________________________________________________________
PROBLEM t = 1


<IPython.core.display.Math object>

__________________________________________________________________
PROBLEM t = 2


...

__________________________________________________________________
PROBLEM t = 5


<IPython.core.display.Math object>

<IPython.core.display.Math object>

___________


<IPython.core.display.Math object>

In [183]:
example_small.check_all( (mu_small, U_small,V_small))

____
<h1>Feasibility</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#matched: 5 over 6

____
<h1>Generalized Complementary Slackness</h1>

...

____
<h1>Individual Rationality</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Big examples

In [184]:
n_big = 5000
m_big = 6000
example_big = OneToOneITU( n = n_big ,m = m_big , size_params= (100,2) , random_seed= 10)

In [185]:
mu_big , U_big,V_big = example_big.find_eq()

In [186]:
example_big.check_all((mu_big,U_big,V_big))

____
<h1>Feasibility</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#matched: 0 over 5000

____
<h1>Generalized Complementary Slackness</h1>

...

____
<h1>Individual Rationality</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Let's try many examples

In [202]:
iters = 100
error_tollerance = 1e-14

n_r = 300
m_r = 300

violations = []
for iter in range(iters):
    print(iter)
    
    example = OneToOneITU( n = n_r,m = m_r, size_params= (2,1) , random_seed= iter**2, tol = .1)

    
    mu_r , U_r, V_r = example.find_eq()
    viol_dual_feas = np.abs(np.mean(np.minimum(U_r[:,None] + example.A_ij * V_r - example.B_ij ,0)))
    viol_dual_CS =  np.mean((U_r - example.lb_U) * ( 1 - np.sum(mu_r, axis = 1))) + np.mean((V_r - example.lb_V) * ( 1 - np.sum(mu_r, axis = 0)))
    viol_prim_CS = np.abs(np.sum((U_r[:,None] + example.A_ij * V_r - example.B_ij) * mu_r))
    viol_prim_feas = np.any(np.sum(mu_r, axis = 1) > 1) * np.any(np.sum(mu_r, axis = 0) > 1)

    # check mu is a binary matrix
    valid_i = np.all(np.logical_or(np.sum(mu_r, axis=0) == 1, np.sum(mu_r, axis=0) == 0))
    valid_j = np.all(np.logical_or(np.sum(mu_r, axis=1) == 1, np.sum(mu_r, axis=1) == 0))
    valid = valid_i * valid_j

    if (-viol_dual_feas + viol_dual_CS + viol_prim_CS)/3 > error_tollerance or viol_prim_feas or not valid:
        print("########")
        print(f"violation in iter #{iter} (if numbers are negligible, it's numerical error)")
        print(f"dual feas: {viol_dual_feas}")
        print(f"dual CS: {viol_dual_CS}")
        print(f"primal CS: {viol_prim_CS}")
        print(f"primal feas : {not viol_prim_feas}")
        violations.append([iter, np.max([viol_dual_feas, viol_prim_CS]) ])
        
    
print("##################################")
print("##################################")
print(f"number of violations: {np.shape(violations)[0]}")
if np.size(violations) > 0:
    print(f"maximum violation: {np.max(np.array(violations)[:,1])}")
print("##################################")
print("##################################")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
##################################
##################################
number of violations: 0
##################################
##################################


### Compare with Gurobi in TU case

In the sut up above, standard TU corresponds to the case with $A \equiv G \equiv 1$ and $U_0 =V_0 = 0.$ $B$ represents the surplus matrix.

In [191]:
n_TU = 30
m_TU = 30
example_TU = OneToOneITU( n = n_TU,m = m_TU, size_params= (100,2) , random_seed= 8, lbs = (0,0), tol = 1e-6 )

example_TU.A_ij = np.ones([n_TU,m_TU])
# example_TU.B_ij = np.random.randint(0,10, size = [n_TU,m_TU])


In [192]:
mu_TU ,U_TU, V_TU = example_TU.find_eq(print_output= False)

In [193]:
example_TU.check_all((mu_TU ,U_TU, V_TU))

____
<h1>Feasibility</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#matched: 30 over 30

____
<h1>Generalized Complementary Slackness</h1>

...

____
<h1>Individual Rationality</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The following function finds the optimal matching solving the assignment problem:

In [194]:
def solve_1to1(Φ_i_j):
    n, m = np.shape(Φ_i_j)
    M_z_a = spr.bmat([[spr.kron(spr.eye(n), np.ones((1, m)))],
                      [spr.kron(np.ones((1, n)), spr.eye(m))]])
  
    q = np.concatenate((np.ones(n), np.ones(m)))

    model = grb.Model()
    mu_a = model.addMVar(n * m, vtype=grb.GRB.INTEGER)
    model.setObjective(Φ_i_j.flatten() @ mu_a, GRB.MAXIMIZE)
    model.addConstr(M_z_a @ mu_a <= q)
    
    model.Params.OutputFlag = 0
    model.optimize()

    return np.array(model.x, dtype= bool).reshape([n,m])*1

In [195]:
mu_gurobi  = solve_1to1(example_TU.B_ij)
(mu_gurobi * example_TU.B_ij).sum()

30

In [196]:
U_TU.sum() + V_TU.sum()

29.999999999999996

In [197]:
(mu_TU * example_TU.B_ij).sum()

30

In [198]:
np.all(mu_TU == mu_gurobi)

False

In [ ]:
n_01 = 100
m_01 = 100
example_01 = OneToOneITU( n = n_01,m = m_01, params=(7,6,2,10), random_seed= 59**3, lbs = (0,0), tol =  0.1)
np.random.seed(19)
example_01.A_ij = np.ones([n_01,m_01])
example_01.G_ij = np.ones([n_01,m_01])
example_01.B_ij = np.random.randint(0,2, size = [n_01,m_01])

In [ ]:
mu_01 ,U_01, V_01 = example_01.find_eq(print_output= False)

In [ ]:
example_01.check_all((mu_01 ,U_01, V_01))

____
<h1>Feasibility</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#matched: 100 over 100

____
<h1>Generalized Complementary Slackness</h1>

...

____
<h1>Individual Rationality</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>